<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Clustering using Vantage and Amazon Bedrock
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style="font-size:16px;font-family:Arial;color:#00233c">This feature uses advanced clustering techniques powered by <b>Teradata Vantage</b> and <b>Amazon Bedrock's Generative AI Embeddings</b> model to group similar customer complaints together. By identifying common themes and patterns, this functionality provides valuable insights into the key issues and pain points experienced by customers.</p>


<p style="font-size:16px;font-family:Arial;color:#00233c"><b>Key Features of Complaints Clustering:</b></p>
<ul style="font-size:16px;font-family:Arial;color:#00233c">
  <li>Leverages advanced clustering algorithms powered by <b>Teradata Vantage</b> and <b>Amazon Bedrock's Generative AI Embeddings.</b></li>
  <li>Groups similar customer complaints together, revealing common themes and pain points.</li>
  <li>Provides clients with a deeper understanding of the key issues affecting their customers.</li>
  <li>Enables clients to prioritize and address the most pressing concerns more effectively.</li>
  <li>Helps clients identify opportunities for product improvements and enhanced customer experience.</li>
</ul>


<p style = 'font-size:16px;font-family:Arial;color:#00233c'>Unlock the revolutionary potential of Generative AI to categorize and analyze complaints with unparalleled efficiency.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
    <li>Configuring AWS CLI</li>
    <li>Cluster the Complaints</li>
    <li>Cleanup</li>
</ol>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Downloading and installing additional software needed</b>

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
!pip install pyopenssl cryptography --force-reinstall --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import numpy as np
import timeit
import tqdm
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import *
tqdm_notebook.pandas()

# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = "whitegrid")

# teradata lib
from teradataml import *
import teradataml
from teradataml import KMeans
from teradataml.analytics.valib import *
from teradataml import configure
from sqlalchemy import func
configure.val_install_location = "val"
configure.byom_install_location = "byom"

# helper functions
# from utils.sql_helper_func import *
from utils.bedorck_helper_func import *
# from utils.opensource_helper_func import *

# genai
import getpass
import os
import boto3
from langchain_community.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
display.max_rows = 5

display.print_sqlmr_query = False
display.suppress_vantage_runtime_warnings = True

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Complaints_Analysis_GenAI.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>

In [ ]:
df = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.1 Graph for Count of Product Complaints Over Years</b></p>

<p style='font-size:16px;font-family:Arial;color:#00233c'>The provided graph visualizes the count of complaints over the past few years, categorized by product names.</p>

In [ ]:
viz_df = df.assign(year = func.td_year_of_calendar(df.date_received.expression))

In [ ]:
pd_df = viz_df.select(['product','year','complaint_id']).groupby(['product', 'year']).agg(['count']).to_pandas()

In [ ]:
# Sorting the DataFrame by year for each product
pd_df_sorted = pd_df.sort_values(by=['product', 'year'])

# Plotting using Plotly
fig = px.line(pd_df_sorted, x='year', y='count_complaint_id', color='product', markers=True, title='Count of Product Complaints Over Years')
fig.update_layout(xaxis_title='Year', yaxis_title='Count', legend_title='Product', width=1200, height=600)

fig.show()

<hr style='height:1px;border:none;background-color:#00233C;'> 
<p style='font-size:18px;font-family:Arial;color:#00233c'><b>2.2 Graph for Count of Complaints by Months</b></p> 
<p style='font-size:16px;font-family:Arial;color:#00233c'>The provided graph visualizes the count of complaints by months. We can see that the mean count is above 500, and the July and August months have the maximum complaints count.</p>

In [ ]:
df = df.assign(complaint_month = func.td_month_of_year(df.date_received.expression))
grp_gen = df.select(['complaint_month','complaint_id']).groupby(['complaint_month']).agg(['count']).to_pandas()

# Define a reverse mapping dictionary
reverse_month_mapping = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
                         7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

# Create a new column with month names based on reverse mapping
grp_gen['month'] = grp_gen['complaint_month'].map(reverse_month_mapping)


fig = px.bar(grp_gen.sort_values(by = 'complaint_month'), x='month', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'month': 'Complaint Month'},
             title='Number of Complaints by Month')

# Add hover information
fig.update_traces(hovertemplate='Month: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style='height:1px;border:none;background-color:#00233C;'> 

<p style='font-size:18px;font-family:Arial;color:#00233c'><b>2.3 Graph for Number of Complaints by Product</b></p> <p style='font-size:16px;font-family:Arial;color:#00233c'>The graph displays the number of complaints received for different products. The data shows that the highest number of complaints are related to credit cards or prepaid cards, as well as credit reporting and credit repair services.</p>

In [ ]:
# Assuming df is your DataFrame and sns is seaborn

grp_gen = df.select(['product','complaint_id']).groupby(['product']).agg(['count']).to_pandas()

fig = px.bar(grp_gen, x='product', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'product': 'Product'},
             title='Number of Complaints by Product')

# Add hover information
fig.update_traces(hovertemplate='Product: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style='height:1px;border:none;background-color:#00233C;'> 

<p style='font-size:18px;font-family:Arial;color:#00233c'><b>2.4 Graph for Number of Complaints by Issue</b></p> <p style='font-size:16px;font-family:Arial;color:#00233c'>The graph displays the number of complaints received for different issues. The data shows that the highest number of complaints are related to issue of incorrect information on your report.</p>

In [ ]:
# Assuming df is your DataFrame and sns is seaborn

grp_gen = df.select(['issue','complaint_id']).groupby(['issue']).agg(['count']).to_pandas()

grp_gen = grp_gen.sort_values('count_complaint_id', ascending = False)[:10]

fig = px.bar(grp_gen, x='issue', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'issue': 'Issue'},
             title='Number of Complaints by Issue(Top 10)')

# Add hover information
fig.update_traces(hovertemplate='Issue: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style='height:1px;border:none;background-color:#00233C;'> 

<p style='font-size:18px;font-family:Arial;color:#00233c'><b>2.5 Graph for Number of Complaints by Sub-Issue</b></p> 

<p style='font-size:16px;font-family:Arial;color:#00233c'>The graph displays the number of complaints received for different sub-issues. The data shows that the highest number of complaints are related to issue of information belongs to someone else.</p>

In [ ]:
# Assuming df is your DataFrame and sns is seaborn

grp_gen = df.select(['sub_issue','complaint_id']).groupby(['sub_issue']).agg(['count']).to_pandas()

grp_gen = grp_gen.sort_values('count_complaint_id', ascending = False)[:10]

fig = px.bar(grp_gen, x='sub_issue', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'sub_issue': 'Sub-Issue'},
             title='Number of Complaints by Sub-Issue(Top 10)')

# Add hover information
fig.update_traces(hovertemplate='Sub-Issue: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style='height:1px;border:none;background-color:#00233C;'> 

<p style='font-size:18px;font-family:Arial;color:#00233c'><b>2.6 Graph for Number of Complaints by Channel</b></p>

<p style='font-size:16px;font-family:Arial;color:#00233c'>The graph displays the number of complaints received for different issues. The data shows that the all the complaints are submitted by web channel.</p>

In [ ]:
grp_gen = df.select(['submitted_via','complaint_id']).groupby(['submitted_via']).agg(['count']).to_pandas()

# Create a mapping of numbers to product names
product_mapping = {i: product for i, product in enumerate(grp_gen['submitted_via'])}

# Replace product names with numbers in the DataFrame
grp_gen['product_num'] = grp_gen['submitted_via'].map({product: i for i, product in enumerate(grp_gen['submitted_via'])})

fig = px.bar(grp_gen, x='submitted_via', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'submitted_via': 'Submitted Via'},
             title='Number of Complaints by Channel')

# Add hover information
fig.update_traces(hovertemplate='Submitted Via: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Configuring AWS CLI</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will prompt us for the following information:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
def configure_aws():
    print("configure the AWS CLI")
    # enter the access_key/secret_key
    access_key = getpass.getpass("aws_access_key_id ")
    secret_key = getpass.getpass("aws_secret_access_key ")
    region_name = getpass.getpass("region name")

    #set to the env
    !aws configure set aws_access_key_id {access_key}
    !aws configure set aws_secret_access_key {secret_key}
    !aws configure set default.region {region_name}

In [ ]:
does_access_key_exists = !aws configure get aws_access_key_id

if len(does_access_key_exists) == 0:
    configure_aws()

In [ ]:
!aws configure list

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cluster the Complaints</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For our complaint clustering task, we'll be using a sample of the data to cluster the complaints. This approach will allow us to effectively analyze and categorize the complaints without using the entire dataset.</p>

In [ ]:
embedded_df = get_embeddings_bedrock(df.to_pandas(), 'consumer_complaint_narrative')
copy_to_sql(embedded_df, table_name ='complaints_embeddings', if_exists = 'replace')

In [ ]:
embedded_df = DataFrame('complaints_embeddings')
embedded_df

In [ ]:
cols = list(embedded_df.columns[:17]) + ['consumer_complaint_narrative']
embedded_df = embedded_df.drop(columns=cols)
copy_to_sql(df = embedded_df, table_name = 'kmeans_features', if_exists = 'replace')

In [ ]:
no_clusters = 5

# drop existing model
try:
    db_drop_table(table_name = "KMeans_Model")
except:
    pass

# Model Table
query = f'''
    SELECT * from TD_KMeans (
    ON kmeans_features as InputTable
    OUT TABLE ModelTable(KMeans_Model)
    USING
        IdColumn('complaint_id')
        TargetColumns{tuple(embedded_df.columns[19:])}
        NumClusters({no_clusters})
        OutputClusterAssignment('true')
) AS dt;
'''

execute_sql(query)

In [ ]:
KMeans_Model = DataFrame("KMeans_Model")
print("Data information: \n", KMeans_Model.shape)

In [ ]:
no_clusters = 5

# for Cluster info, used later for filtering
query = f'''
    SELECT * from TD_KMeans (
    ON kmeans_features as InputTable
    USING
        IdColumn('complaint_id')
        TargetColumns{tuple(embedded_df.columns[19:])}
        NumClusters({no_clusters})
        OutputClusterAssignment('true')
) AS dt;
'''

kmeans_out_sql = DataFrame.from_query(query)

In [ ]:
kmeans_out_sql

In [ ]:
embeddings_cluster = DataFrame('complaints_embeddings').join(other=kmeans_out_sql, how="full", on="complaint_id=complaint_id", lprefix= "L_")

In [ ]:
# View complaints in cluster 1
embeddings_cluster[['td_clusterid_kmeans','complaint_id','consumer_complaint_narrative']].loc[embeddings_cluster.td_clusterid_kmeans==1]

<hr style='height:1px;border:none;background-color:#00233C;'> 

<p style='font-size:18px;font-family:Arial;color:#00233c'><b>4.1 Visualization of Clusters with Complaints</b></p> 

<p style='font-size:16px;font-family:Arial;color:#00233c'>The graph displays the clustering of complaints into distinct groups. Based on the analysis, the data has been divided into 5 optimal clusters, each representing a unique pattern or category of complaints. This clustering approach helps to identify the key areas or types of complaints that are most prevalent, allowing for more targeted investigation and resolution efforts.</p>

In [ ]:
# emb = DataFrame('kmeans_features').to_pandas()
clus = embeddings_cluster.to_pandas()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(clus.iloc[:, 20:-1])

In [ ]:
tsne_df = pd.DataFrame(tsne_result, columns=['tsne_1', 'tsne_2'])
tsne_df['cluster_id'] = clus['td_clusterid_kmeans']
tsne_df['complaint_id'] = clus['consumer_complaint_narrative']

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have already computed tsne_df as per the previous example

# Create a new DataFrame combining t-SNE results with complaint information
tsne_complaint_df = pd.DataFrame(tsne_result, columns=['tsne_1', 'tsne_2'])
tsne_complaint_df['cluster_id'] = clus['td_clusterid_kmeans']
tsne_complaint_df['complaint_id'] = clus['complaint_id']
tsne_complaint_df['complaint'] = clus['consumer_complaint_narrative']

# Truncate text for hover data
max_chars = 50  # Maximum characters to display
tsne_complaint_df['truncted_complaint'] = clus['consumer_complaint_narrative'].apply(lambda x: x[:max_chars] + '...' if len(x) > max_chars else x)

# Plot using Plotly Express
fig = px.scatter(tsne_complaint_df, x='tsne_1', y='tsne_2', color='cluster_id',
                 hover_data=['complaint_id', 'truncted_complaint', 'cluster_id'])

fig.update_traces(marker=dict(size=15))
fig.update_layout(
    title='t-SNE Visualization of Clusters with Complaints',
    xaxis_title='dimension-1',
    yaxis_title='dimension-2',
    xaxis=dict(tickangle=45),
    width=1000,
    height=800,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    ),
    autosize=False,
)

# Customize the hovertemplate
fig.update_traces(hovertemplate="<b>Complaint ID:</b> %{customdata[0]}<br>"
                                 "<b>Complaint:</b> %{customdata[1]}<br>"
                                 "<b>Cluster ID:</b> %{customdata[2]}<br><extra></extra>")

fig.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>